## Circuit cutting with automatic cut finding using the Circuit Knitting Toolbox

### Import relevant modules

In [5]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Options,
    Session,
    Sampler,
    RuntimeOptions,
)

from circuit_knitting_toolbox.circuit_cutting import WireCutter

### Create a circuit to cut

In [6]:
qc = QuantumCircuit(5)
for i in range(5):
    qc.h(i)
qc.cx(0, 1)
for i in range(2, 5):
    qc.t(i)
qc.cx(0, 2)
qc.rx(np.pi / 2, 4)
qc.rx(np.pi / 2, 0)
qc.rx(np.pi / 2, 1)
qc.cx(2, 4)
qc.t(0)
qc.t(1)
qc.cx(2, 3)
qc.ry(np.pi / 2, 4)
for i in range(5):
    qc.h(i)

qc.draw()

┌───┐                ┌─────────┐   ┌───┐   ┌───┐
q_0: ┤ H ├──■───────■─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤┌─┴─┐     │     ├─────────┤   ├───┤   ├───┤
q_1: ┤ H ├┤ X ├─────┼─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤├───┤   ┌─┴─┐   └─────────┘   └───┘   ├───┤
q_2: ┤ H ├┤ T ├───┤ X ├────────■──────────■─────┤ H ├
     ├───┤├───┤   └───┘        │        ┌─┴─┐   ├───┤
q_3: ┤ H ├┤ T ├────────────────┼────────┤ X ├───┤ H ├
     ├───┤├───┤┌─────────┐   ┌─┴─┐   ┌──┴───┴──┐├───┤
q_4: ┤ H ├┤ T ├┤ Rx(π/2) ├───┤ X ├───┤ Ry(π/2) ├┤ H ├
     └───┘└───┘└─────────┘   └───┘   └─────────┘└───┘

### Set up the Qiskit runtime session

In [7]:
service_args = QiskitRuntimeService(
    channel="ibm_quantum",
    token="<YOUR_API_KEY>",
).active_account()

### Cut the circuit and evaluate the subcircuits within a Qiskit Session context

In [8]:
# Set the Sampler options
options = Options(resilience_level=1, optimization_level=3, execution={"shots": 8192})
runtime_options = RuntimeOptions(backend="ibmq_qasm_simulator")

# Instantiate a WireCutter and decompose the circuit
cutter = WireCutter(
    qc, service_args=service_args, options=options, runtime_options=runtime_options
)
# cutter = WireCutter(qc) # Local estimator

cuts = cutter.decompose(method="manual", subcircuit_vertices=[[0, 1], [2, 3]])

2022-10-08 18:37:30,455	INFO worker.py:1518 -- Started a local Ray instance.


(_cut_manual pid=10547) --------------------
(_cut_manual pid=10547) subcircuit 0
(_cut_manual pid=10547) ρ qubits = 0, O qubits = 1, width = 3, effective = 2, depth = 6, size = 12
(_cut_manual pid=10547)      ┌───┐                     ┌─────────┐┌───┐┌───┐
(_cut_manual pid=10547) q_0: ┤ H ├──■───────────────■──┤ Rx(π/2) ├┤ T ├┤ H ├
(_cut_manual pid=10547)      ├───┤┌─┴─┐┌─────────┐  │  └──┬───┬──┘├───┤└───┘
(_cut_manual pid=10547) q_1: ┤ H ├┤ X ├┤ Rx(π/2) ├──┼─────┤ T ├───┤ H ├─────
(_cut_manual pid=10547)      ├───┤├───┤└─────────┘┌─┴─┐   └───┘   └───┘     
(_cut_manual pid=10547) q_2: ┤ H ├┤ T ├───────────┤ X ├─────────────────────
(_cut_manual pid=10547)      └───┘└───┘           └───┘                     
(_cut_manual pid=10547) subcircuit 1
(_cut_manual pid=10547) ρ qubits = 1, O qubits = 0, width = 3, effective = 3, depth = 6, size = 11
(_cut_manual pid=10547)                                           ┌───┐
(_cut_manual pid=10547) q_0: ───────────────────────■───────■─────┤ H ├


### Recompose the circuit and verify the error between the full and cut circuit distributions is within tolerance

In [9]:
# Evaluate the subcircuits on backend
subcircuit_instance_probabilities = cutter.evaluate(cuts)

# Recompose the circuit and generate the cut circuit's probability distribution
reconstructed_probabilities = cutter.recompose(
    subcircuit_instance_probabilities, cuts, num_threads=4
)

# Use a statevector simulator to calculate the error between the inferred and actual distributions
metrics = cutter.verify(reconstructed_probabilities)
print(metrics)

{'nearest': {'chi2': 0.00172008668779893, 'Mean Squared Error': 2.8792313482165644e-06, 'Mean Absolute Percentage Error': 8.1461957348589, 'Cross Entropy': 2.601394920803805, 'HOP': 0.8965535804235953}, 'naive': {'chi2': 0.00172008668779893, 'Mean Squared Error': 2.8792313482165644e-06, 'Mean Absolute Percentage Error': 8.1461957348589, 'Cross Entropy': 2.601394920803805, 'HOP': 0.8965535804235953}}
